In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import one_hot

In [ ]:
df_train = pd .read_csv('/content/drive/MyDrive/Colab Notebooks/fake news classifier/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/fake news classifier/test.csv')

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(len(df_train)):
  title = re.sub('[^a-zA-Z]', ' ', str(df_train['title'][i]))
  title = title.lower()
  title = title.split()
  ps = PorterStemmer()
  title = [ps.stem(word) for word in title if not word in set(stopwords.words('english'))]
  title = ' '.join(title)
  corpus.append(title)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
voc_size = 10000
onehot_words = [one_hot(words, voc_size) for words in corpus]

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense,LSTM, Bidirectional, Dropout

In [ ]:
max_len = 40
emb_sen = pad_sequences(onehot_words, padding='pre', maxlen=max_len)

In [8]:
model = Sequential()
model.add(Embedding(voc_size, 100, input_length=max_len))
model.add(LSTM(100))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
model1 = Sequential()
model1.add(Embedding(voc_size, 100, input_length=max_len))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [12]:
import numpy as np
x = np.array(emb_sen)
y = np.array(df_train['label'])

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [17]:
model1.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=64)

Epoch 1/20
260/260 [==============================] - 14s 20ms/step - loss: 0.2650 - accuracy: 0.8879 - val_loss: 0.1733 - val_accuracy: 0.9281
Epoch 2/20
260/260 [==============================] - 4s 17ms/step - loss: 0.1036 - accuracy: 0.9603 - val_loss: 0.1858 - val_accuracy: 0.9257
Epoch 3/20
260/260 [==============================] - 4s 17ms/step - loss: 0.0594 - accuracy: 0.9800 - val_loss: 0.2213 - val_accuracy: 0.9274
Epoch 4/20
260/260 [==============================] - 4s 17ms/step - loss: 0.0304 - accuracy: 0.9900 - val_loss: 0.2858 - val_accuracy: 0.9231
Epoch 5/20
260/260 [==============================] - 4s 17ms/step - loss: 0.0176 - accuracy: 0.9950 - val_loss: 0.3354 - val_accuracy: 0.9171
Epoch 6/20
260/260 [==============================] - 4s 17ms/step - loss: 0.0096 - accuracy: 0.9969 - val_loss: 0.4038 - val_accuracy: 0.9224
Epoch 7/20
260/260 [==============================] - 4s 17ms/step - loss: 0.0057 - accuracy: 0.9983 - val_loss: 0.3966 - val_accuracy: 0.917

In [18]:
pred = model1.predict_classes(x_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.90625

In [20]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus_test = []
for i in range(len(df_test)):
  title = re.sub('[^a-zA-Z]', ' ', str(df_test['title'][i]))
  title = title.lower()
  title = title.split()
  ps = PorterStemmer()
  title = [ps.stem(word) for word in title if not word in set(stopwords.words('english'))]
  title = ' '.join(title)
  corpus_test.append(title)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
onehot_words_test = [one_hot(words, voc_size) for words in corpus_test]
emb_sen_test = pad_sequences(onehot_words_test, padding='pre', maxlen=max_len)
x_test = np.array(emb_sen_test)
prediction = model.predict(x_test)
prediction = model.predict(x_test)
prediction = prediction.flatten()

In [22]:
submission = pd.DataFrame({
    "id" : df_test["id"],
    "label" : prediction
})
submission.to_csv("/content/drive/MyDrive/Colab Notebooks/fake news classifier/result_label_bidirectional_lstm.csv", index=False)